#Блок импорта библиотек

In [16]:
import pandas as pd #для массовой манипуляции с данными
import seaborn as sns # для построения графиков
import os #Для указания директории файла
import numpy as np #для матемаических функций и операций
import scipy as sk #для функции интерполяции
import re #для регулярных выражений
from datetime import datetime #для преобразования формата datetime
import matplotlib.pyplot as plt #sns использует matplotlib внутри, библиотека нужна для регулирования размеров графика
sns.set(rc={'figure.figsize' : (20,10)}, style='whitegrid') #Добавление сетки и указание размеров графика (ширина, высота)

Using matplotlib backend: TkAgg


#Расчёт максимальной суточной добычи

In [17]:
#Шаг 1 - расчёт пикового дебита за год (пр. (суточная добыча на момент 22.01.2022 - суточная добыча на момент 01.01.2022)/22)
#Шаг 2 - Построить линию тренда, где Y - результат Шага 1, X - Накопленная добыча на момент даты 22.10.2022 (применить полиномиальную функцию к расчёту)
#Шаг 3 - Добавить возможность ручного ввода годовой накопленной добычи, таким образом конечный пользователь сможет видеть зависимость линии тренда от времени
#!!!Задание - организовать проверку физичности для пикового дебита, предыдущий всегда больше, чем последующие
def max_sut(max_sut):
    pd.options.display.float_format = '{:.2f}'.format
    
    
    
    
    
    
    
    
    return max_sut

#Расчёт добычи по кварталам
Функция возвращает датафрейм с расчитанными годовыми значениями добычи
Расчёт выполняется для каждого уникального типа отдельно итеративным способом

In [18]:
def quart(df_quart, n):
    pd.options.display.float_format = '{:.2f}'.format
    # df_quart = df_quart[(df_quart['type'].str.contains('1')) ]
    unique = df_quart['type'].unique()
    print(len(unique))
    df_conc=[]
    for m in range(0, len(unique)): # цикл для перебора уникальных типов
        df_quart1 = df_quart[(df_quart['type'] == unique[m]) ]
        # print(df_quart1)
        df_temp = df_quart.groupby(['Год', 'Квартал'], as_index=False).agg({"Дата": [np.min]})
        # print(df_temp)
        df_temp.columns = df_temp.columns.droplevel(1)
        df_temp = df_temp.merge(df_quart1[['Накопленная добыча газа, м3', 'Дата']], how='inner', left_on='Дата', right_on='Дата')
        df_temp['Квартальная добыча'] = df_temp['Накопленная добыча газа, м3'] - df_temp['Накопленная добыча газа, м3'].shift(1)# от каждого последующего значения накопленной добычи отнимается предыдущее в рамках квартала
        df_temp['type'] = unique[m]
        df_conc.append(df_temp) 
    df_conc = pd.concat(df_conc, ignore_index=True)
    # print(df_conc)
    return df_conc

#Основной блок
В шаблоне ОБЯЗАТЕЛЬНО(!!!) должны присутствовать два листа:
1) Лист с данными, сюда входят все 8 синтетических вариантов (Название листа по умолчанию 'Вариант 1')
2) Лист с перечнем условий, программа будет динамически расширять столбцы для новых условий, их можно добавить любое количество, однако, важно чтобы для этих условий были данные (Название листа с условиями по умолчанию 'Условия')
ОБЯЗАТЕЛЬНО(!!!) также наличие поля type (в десятичном формате), type характеризует текущий набор данных (условия)

In [19]:
cwd = os.getcwd() #определение директории файла
df=pd.read_excel(os.path.join(cwd, 'test_data_s.xlsx'), sheet_name='Вариант 1') #считывание данных из файла, формировнаие основного датафрейма
df_usl=pd.read_excel(os.path.join(cwd, 'test_data_s.xlsx'), sheet_name='Условия') #считывание данных из файла, формирование датафрейма условий

#определение квартала и года по дате
df['Квартал'] = df['Дата'].dt.quarter 
df['Год'] = df['Дата'].dt.year

df['type'] = df['type'].apply(lambda x: format(int(x), '04b')) #перевод десятичных значений поля 'type' в байты

n=len(df_usl.index) #максимальное количество условий для одного варианта

#цикл для удаления первых нулей в полях 'type' (пр. для '7': 0111 > 111)
for i in range(0, len(df.index)):
    typ_st = df['type'][i]
    if len(df['type'][i]) > n and int(typ_st[0]) == 0:
        df['type'][i] = typ_st[1:]
# print(df)

#формирование квартала и года из даты посредством регулярных выражений
df_usl['Квартал реализации условия'] = df_usl['Дата условия'].str.extract(r'(?!\()\b(\d+){1}') 
df_usl['Год реализации условия'] = df_usl['Дата условия'].str.extract(r'(\d{4})')



name='C'
for numbers in range(0, len(df_usl.index)): #Цикл для формирования столбцов-условий в главном датафрейме
    df[name+str(numbers)] = np.nan

#Формирование побочных датафреймов для их обработке внутри функций расчёта
df_quart = df
max_sut = df

#Цикл для заполнения столбцов-условий в главном датафрейме
for i in range(0, len(df.index)): #перебор строк датафрейма с данными
    # print(df['Дата'][1])
    for j in range(0, len(df_usl.index)): #перебор строк датафрейма условий
        st_t=str(df['type'][i])
        # print(str(df['Квартал'][i])==str(df_usl['Квартал реализации условия'][j]) and str(df['Год'][i])==str(df_usl['Год реализации условия'][j]))
        if str(df['Квартал'][i])==str(df_usl['Квартал реализации условия'][j]) and str(df['Год'][i])==str(df_usl['Год реализации условия'][j]): #поиск строк с условиями 
            for h in range(0, n): #для перебора байтов и записи столбцов 'С'
                if int(st_t[h]) == 1:
                    df['C'+str(h)][i] = df_usl['Дата условия'][h]
                elif int(st_t[h]) == 0:
                    df['C'+str(h)][i] = np.nan

df_quart=quart(df_quart, n) # Вызов функций 
max_sut=(max_sut)


# print(df_quart)

#График зависимости годовой добычи от времени
sns.lineplot(
    data= df_quart[df_quart['Дата'] >= '2022-04-01'], #условие с датой необходимо, первые значения датафрейма невалидны, в будущем будет сделана динамическое исключение таких значений
    x='Дата',
    y='Квартальная добыча',
    hue='type',
)


# plt.figure(figsize=(20, 10))
# df_quart[df_quart['Квартальная добыча'].isna()]
# df_quart.head(20)
# df_quart[df_quart.type == '101']




C:\Users\A_Korobeynikov\AppData\Local\Temp\ipykernel_10404\2155181855.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'][i] = typ_st[1:]
C:\Users\A_Korobeynikov\AppData\Local\Temp\ipykernel_10404\2155181855.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['C'+str(h)][i] = df_usl['Дата условия'][h]


9


<AxesSubplot:xlabel='Дата', ylabel='Квартальная добыча'>